In [ ]:
# 4-2 KNN
import polars as pl
from sklearn.neighbors import NearestNeighbors  # k-NN

k_in_knn = 5  # k-NN における k

dataset = pl.read_csv("../test_data/resin.csv")
x_prediction = pl.read_csv("../test_data/resin_prediction.csv")

# データ分割
y = dataset.get_column("property")  # 目的変数
x = dataset.drop(y.name)  # 説明変数

# 標準偏差が 0 の特徴量の削除
deleting_variables = [
    col for col, std in x.std().row(0, named=True).items() if std in [0, None]
]
x = x.drop(deleting_variables)
x_prediction = x_prediction.drop(deleting_variables)

# オートスケーリング
autoscaled_x = x.select((pl.all() - pl.all().mean()) / pl.all().std())
autoscaled_x_prediction = (
    x_prediction - x.mean().select(pl.all().repeat_by(x_prediction.height).explode())
) / x.std().select(pl.all().repeat_by(x_prediction.height).explode())

# k-NN による AD
ad_model = NearestNeighbors(n_neighbors=k_in_knn, metric="euclidean")  # AD モデルの宣言
ad_model.fit(
    autoscaled_x
)  # k-NN による AD では、トレーニングデータの x を model_ad に格納することに対応

# サンプルごとの k 最近傍サンプルとの距離に加えて、k 最近傍サンプルのインデックス番号も一緒に出力されるため、出力用の変数を 2 つに
# トレーニングデータでは k 最近傍サンプルの中に自分も含まれ、自分との距離の 0 を除いた距離を考える必要があるため、k_in_knn + 1 個と設定
knn_distance_train, knn_index_train = ad_model.kneighbors(
    autoscaled_x, n_neighbors=k_in_knn + 1
)
test = pl.DataFrame(knn_distance_train).drop(pl.first())
mean_of_knn_distance_train = (
    # 自分以外の k_in_knn 個の距離の平均
    pl.DataFrame(knn_distance_train)
    .drop(pl.first())
    .mean_horizontal()
    .alias("mean_of_knn_distance")
)
mean_of_knn_distance_train.to_frame().write_csv(
    "../output/mean_of_knn_distance_train.csv"
)

# AD 内となるトレーニングデータの割合。ADのしきい値を決めるときに使用
rate_of_training_samples_inside_ad = 0.96
# トレーニングデータのサンプルの rate_of_training_samples_inside_ad * 100 % が含まれるようにしきい値を設定
ad_threshold: float = mean_of_knn_distance_train.sort().item(
    # サンプル数の96%の位置の値を閾値とする
    round(autoscaled_x.height * rate_of_training_samples_inside_ad) - 1
)

# トレーニングデータに対して、AD の中か外かを判定
inside_ad_flag_train = mean_of_knn_distance_train.le(ad_threshold).alias(
    "inside_ad_flag"
)
inside_ad_flag_train.to_frame().write_csv("../output/inside_ad_flag_train_knn.csv")

# 予測用データに対する k-NN 距離の計算
knn_distance_prediction, knn_index_prediction = ad_model.kneighbors(
    autoscaled_x_prediction
)
# k_in_knn 個の距離の平均
mean_of_knn_distance_prediction = (
    pl.DataFrame(knn_distance_prediction)
    .mean_horizontal()
    .alias("mean_of_knn_distance")
)
mean_of_knn_distance_prediction.to_frame().write_csv(
    "../output/mean_of_knn_distance_prediction.csv"
)

# 予測用データに対して、AD の中か外かを判定
inside_ad_flag_prediction = mean_of_knn_distance_prediction.le(ad_threshold)
inside_ad_flag_prediction.to_frame().write_csv(
    "../output/inside_ad_flag_prediction_knn.csv"
)

In [8]:
# 4-2 One-Class Support Vector Machine

from sklearn.svm import OneClassSVM

ocsvm_nu = 0.04  # OCSVM における ν。トレーニングデータにおけるサンプル数に対する、サポートベクターの数の下限の割合
ocsvm_gamma = 0.1  # OCSVM における γ

dataset = pl.read_csv("../test_data/resin.csv")
x_prediction = pl.read_csv("../test_data/resin_prediction.csv")

# データ分割
y = dataset.get_column("property")  # 目的変数
x = dataset.drop(y.name)  # 説明変数

# 標準偏差が 0 の特徴量の削除
deleting_variables = [
    col for col, std in x.std().row(0, named=True).items() if std in [0, None]
]
x = x.drop(deleting_variables)
x_prediction = x_prediction.drop(deleting_variables)

# オートスケーリング
autoscaled_x = x.select((pl.all() - pl.all().mean()) / pl.all().std())
autoscaled_x_prediction = (
    x_prediction - x.mean().select(pl.all().repeat_by(x_prediction.height).explode())
) / x.std().select(pl.all().repeat_by(x_prediction.height).explode())

# OCSVM による AD
ad_model = OneClassSVM(kernel="rbf", gamma=ocsvm_gamma, nu=ocsvm_nu)  # AD モデルの宣言
ad_model.fit(autoscaled_x)  # モデル構築

# トレーニングデータのデータ密度 (f(x) の値)
data_density_train = pl.Series(ad_model.decision_function(autoscaled_x)).alias(
    "ocsvm_data_density"
)
data_density_train.to_frame().write_csv("../output/ocsvm_data_density_train.csv")
number_of_support_vectors = len(ad_model.support_)
number_of_outliers_in_training_data = data_density_train.lt(0).sum()
print(
    f"\nトレーニングデータにおけるサポートベクター数 :{number_of_support_vectors}"
    f"\nトレーニングデータにおけるサポートベクターの割合 :{number_of_support_vectors / x.height}"
    f"\nトレーニングデータにおける外れサンプル数 :{number_of_outliers_in_training_data}"
    f"\nトレーニングデータにおける外れサンプルの割合 :{number_of_outliers_in_training_data / x.height}"
)

# トレーニングデータに対して、AD の中か外かを判定
data_density_train.ge(0).alias("inside_ad_flag").to_frame().write_csv(
    "../output/inside_ad_flag_train_ocsvm.csv"
)

# 予測用データセットのデータ密度 (f(x) の値)
data_density_prediction = pl.Series(
    ad_model.decision_function(autoscaled_x_prediction)
).alias("ocsvm_data_density")
data_density_prediction.to_frame().write_csv(
    "../output/ocsvm_data_density_prediction.csv"
)
number_of_outliers_in_prediction_data = data_density_prediction.lt(0).sum()
print(
    f"\n予測用データセットにおける外れサンプル数 :{number_of_outliers_in_prediction_data}",
    f"\n予測用データセットにおける外れサンプルの割合 :{number_of_outliers_in_prediction_data / x_prediction.height}",
)

# 予測用データセットに対して、AD の中か外かを判定
data_density_prediction.ge(0).alias("inside_ad_flag").to_frame().write_csv(
    "../output/inside_ad_flag_prediction_ocsvm.csv"
)


トレーニングデータにおけるサポートベクター数 :10
トレーニングデータにおけるサポートベクターの割合 :0.5
トレーニングデータにおける外れサンプル数 :4
トレーニングデータにおける外れサンプルの割合 :0.2

予測用データセットにおける外れサンプル数 :4757 
予測用データセットにおける外れサンプルの割合 :0.5826800587947085


In [7]:
# 4-2 One-Class Support Vector Machine Gamma optimization

import polars as pl
from sklearn.svm import OneClassSVM

dataset = pl.read_csv("../test_data/resin.csv")
x_prediction = pl.read_csv("../test_data/resin_prediction.csv")

# データ分割
y = dataset.get_column("property")  # 目的変数
x = dataset.drop(y.name)  # 説明変数

# 標準偏差が 0 の特徴量の削除
deleting_variables = [
    col for col, std in x.std().row(0, named=True).items() if std in [0, None]
]
x = x.drop(deleting_variables)
x_prediction = x_prediction.drop(deleting_variables)

# オートスケーリング
autoscaled_x = x.select((pl.all() - pl.all().mean()) / pl.all().std())
autoscaled_x_prediction = (
    x_prediction - x.mean().select(pl.all().repeat_by(x_prediction.height).explode())
) / x.std().select(pl.all().repeat_by(x_prediction.height).explode())

# 分散最大化によるガウシアンカーネルのγの最適化
ocsvm_gammas = 2.0 ** pl.arange(-20, 11, eager=True)  # OCSVM における γ の候補
autoscaled_x_per_sample = autoscaled_x.select(pl.concat_list(pl.all()).alias("sample"))
sample_distances = (
    # すべてのサンプルを組み合わせる
    autoscaled_x_per_sample.join(autoscaled_x_per_sample, how="cross", suffix="_right")
    .select(
        # 全サンプル間の全データのユークリッド距離の二乗を計算
        (pl.col("sample") - pl.col("sample_right"))
        .list.eval(pl.element().pow(2))
        .list.sum()
        .implode()  # グラム行列の計算を簡単にするために、全距離をリスト化
        .alias("distance"),
    )
    .to_series(0)
)

gram_matrix = (-ocsvm_gammas * sample_distances).list.eval(pl.element().exp())
optimal_gamma: float = ocsvm_gammas.item(gram_matrix.list.var().arg_max())
print("最適化された gamma :", optimal_gamma)

# OCSVM による AD
ocsvm_nu = 0.04  # OCSVM における ν。トレーニングデータにおけるサンプル数に対する、サポートベクターの数の下限の割合
# AD モデルの宣言
ad_model = OneClassSVM(kernel="rbf", gamma=optimal_gamma, nu=ocsvm_nu)
ad_model.fit(autoscaled_x)  # モデル構築

# トレーニングデータのデータ密度 (f(x) の値)
data_density_train = pl.Series(ad_model.decision_function(autoscaled_x)).alias(
    "ocsvm_data_density"
)
data_density_train.to_frame().write_csv(
    "../output/ocsvm_gamma_optimization_data_density_train.csv"
)
number_of_support_vectors = len(ad_model.support_)
number_of_outliers_in_training_data = data_density_train.lt(0).sum()
print(
    f"\nトレーニングデータにおけるサポートベクター数 :{number_of_support_vectors}"
    f"\nトレーニングデータにおけるサポートベクターの割合 :{number_of_support_vectors / x.height}"
    f"\nトレーニングデータにおける外れサンプル数 :{number_of_outliers_in_training_data}"
    f"\nトレーニングデータにおける外れサンプルの割合 :{number_of_outliers_in_training_data / x.height}"
)

# トレーニングデータに対して、AD の中か外かを判定
data_density_train.ge(0).alias("inside_ad_flag").to_frame().write_csv(
    "../output/inside_ad_flag_train_ocsvm_gamma_optimization.csv"
)

# 予測用データセットのデータ密度 (f(x) の値)
data_density_prediction = pl.Series(
    ad_model.decision_function(autoscaled_x_prediction)
).alias("ocsvm_data_density")
data_density_prediction.to_frame().write_csv(
    "../output/ocsvm_gamma_optimization_data_density_prediction.csv"
)
number_of_outliers_in_prediction_data = data_density_prediction.lt(0).sum()
print(
    f"\n予測用データセットにおける外れサンプル数 :{number_of_outliers_in_prediction_data}",
    f"\n予測用データセットにおける外れサンプルの割合 :{number_of_outliers_in_prediction_data / x_prediction.height}",
)

# 予測用データセットに対して、AD の中か外かを判定
data_density_prediction.ge(0).alias("inside_ad_flag").to_frame().write_csv(
    "../output/inside_ad_flag_prediction_ocsvm_gamma_optimization.csv"
)

最適化された gamma : 0.25

トレーニングデータにおけるサポートベクター数 :14
トレーニングデータにおけるサポートベクターの割合 :0.7
トレーニングデータにおける外れサンプル数 :6
トレーニングデータにおける外れサンプルの割合 :0.3

予測用データセットにおける外れサンプル数 :6252 
予測用データセットにおける外れサンプルの割合 :0.7658010779029887


In [ ]:
# 4-3 アンサンブルSVR

import pickle

import numpy as np
import polars as pl
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.svm import SVR

number_of_sub_datasets = 30  # サブデータセットの数
fold_number = 10  # N-fold CV の N

svr_cs = 2 ** np.arange(-5, 11, dtype=float)  # C の候補
svr_epsilons = 2 ** np.arange(-10, 1, dtype=float)  # ε の候補
svr_gammas = 2 ** np.arange(-20, 11, dtype=float)  # γ の候補

dataset = pl.read_csv("../test_data/resin.csv")
x_prediction = pl.read_csv("../test_data/resin_prediction.csv")

# データ分割
y = dataset.get_column("property")  # 目的変数
x = dataset.drop(y.name)  # 説明変数

# 標準偏差が 0 の特徴量の削除
deleting_variables = [
    col for col, std in x.std().row(0, named=True).items() if std in [0, None]
]
x = x.drop(deleting_variables)
x_prediction = x_prediction.drop(deleting_variables)

# オートスケーリング
autoscaled_x = x.select((pl.all() - pl.all().mean()) / pl.all().std())
autoscaled_x_prediction = (
    x_prediction - x.mean().select(pl.all().repeat_by(x_prediction.height).explode())
) / x.std().select(pl.all().repeat_by(x_prediction.height).explode())
autoscaled_y = (y - y.mean()) / y.std()

rate_of_selected_x_variables = (
    0.75  # 各サブデータセットで選択される説明変数の数の割合。0 < x < 1
)
number_of_x_variables = int(np.ceil(x.width * rate_of_selected_x_variables))
print("各サブデータセットにおける説明変数の数 :", number_of_x_variables)
estimated_y_train_all = []  # サブデータセットごとの y の推定結果を追加
selected_x_variable_numbers = []  # 各サブデータセットの説明変数の番号を追加
submodels = []  # 構築済みの各サブモデルを追加

for submodel_number in range(number_of_sub_datasets):
    print(submodel_number + 1, "/", number_of_sub_datasets)  # 進捗状況の表示
    # 説明変数の選択
    # 0 から 1 までの間に一様に分布する乱数を説明変数の数だけ生成して、その乱数値が小さい順に説明変数を選択
    random_x_variables = np.random.rand(x.width)
    selected_x_variable_numbers_tmp = random_x_variables.argsort()[
        :number_of_x_variables
    ]
    selected_x_columns = [x.columns[i] for i in selected_x_variable_numbers_tmp]
    selected_autoscaled_x = autoscaled_x.select(selected_x_columns)
    selected_x_variable_numbers.append(selected_x_variable_numbers_tmp)

    # ハイパーパラメータの最適化
    # 分散最大化によるガウシアンカーネルのγの最適化
    variance_of_gram_matrix = []
    selected_autoscaled_x_array = selected_autoscaled_x.to_numpy()
    for nonlinear_svr_gamma in svr_gammas:
        gram_matrix = np.exp(
            -nonlinear_svr_gamma
            * (
                (
                    selected_autoscaled_x_array[:, np.newaxis]
                    - selected_autoscaled_x_array
                )
                ** 2
            ).sum(axis=2)
        )
        variance_of_gram_matrix.append(gram_matrix.var(ddof=1))
    optimal_svr_gamma = svr_gammas[
        np.where(variance_of_gram_matrix == np.max(variance_of_gram_matrix))[0][0]
    ]
    cross_validation = KFold(
        n_splits=fold_number, shuffle=True
    )  # クロスバリデーションの分割の設定

    # CV による ε の最適化
    r2cvs = []  # 空の list。候補ごとに、クロスバリデーション後の r2 を入れていきます
    for svr_epsilon in svr_epsilons:
        model = SVR(kernel="rbf", C=3, epsilon=svr_epsilon, gamma=optimal_svr_gamma)
        autoscaled_estimated_y_in_cv = cross_val_predict(
            model,
            selected_autoscaled_x.to_numpy(),
            autoscaled_y.to_numpy(),
            cv=cross_validation,
        )
        r2cvs.append(
            r2_score(y.to_numpy(), autoscaled_estimated_y_in_cv * y.std() + y.mean())
        )
    optimal_svr_epsilon = svr_epsilons[
        np.where(r2cvs == np.max(r2cvs))[0][0]
    ]  # クロスバリデーション後の r2 が最も大きい候補

    # CV による C の最適化
    r2cvs = []  # 空の list。候補ごとに、クロスバリデーション後の r2 を入れていきます
    for svr_c in svr_cs:
        model = SVR(
            kernel="rbf", C=svr_c, epsilon=optimal_svr_epsilon, gamma=optimal_svr_gamma
        )
        autoscaled_estimated_y_in_cv = cross_val_predict(
            model,
            selected_autoscaled_x.to_numpy(),
            autoscaled_y.to_numpy(),
            cv=cross_validation,
        )
        r2cvs.append(
            r2_score(y.to_numpy(), autoscaled_estimated_y_in_cv * y.std() + y.mean())
        )
    optimal_svr_c = svr_cs[
        np.where(r2cvs == np.max(r2cvs))[0][0]
    ]  # クロスバリデーション後の r2 が最も大きい候補

    # CV による γ の最適化
    r2cvs = []  # 空の list。候補ごとに、クロスバリデーション後の r2 を入れていきます
    for svr_gamma in svr_gammas:
        model = SVR(
            kernel="rbf", C=optimal_svr_c, epsilon=optimal_svr_epsilon, gamma=svr_gamma
        )
        autoscaled_estimated_y_in_cv = cross_val_predict(
            model,
            selected_autoscaled_x.to_numpy(),
            autoscaled_y.to_numpy(),
            cv=cross_validation,
        )
        r2cvs.append(
            r2_score(y.to_numpy(), autoscaled_estimated_y_in_cv * y.std() + y.mean())
        )
    optimal_svr_gamma = svr_gammas[
        np.where(r2cvs == np.max(r2cvs))[0][0]
    ]  # クロスバリデーション後の r2 が最も大きい候補

    # SVR
    submodel = SVR(
        kernel="rbf",
        C=optimal_svr_c,
        epsilon=optimal_svr_epsilon,
        gamma=optimal_svr_gamma,
    )  # モデルの宣言
    submodel.fit(
        selected_autoscaled_x.to_numpy(), autoscaled_y.to_numpy()
    )  # モデルの構築
    submodels.append(submodel)

# サブデータセットの説明変数の種類やサブデータセットを用いて構築されたモデルを保存
with open("../output/selected_x_variable_numbers_svr_gaussian.bin", "wb") as f:
    pickle.dump(selected_x_variable_numbers, f)
with open("../output/submodels_svr_gaussian.bin", "wb") as f:
    pickle.dump(submodels, f)

# サブデータセットの説明変数の種類やサブデータセットを用いて構築されたモデルを読み込み
with open("../output/selected_x_variable_numbers_svr_gaussian.bin", "rb") as f:
    selected_x_variable_numbers = pickle.load(f)
with open("../output/submodels_svr_gaussian.bin", "rb") as f:
    submodels = pickle.load(f)

# 予測用データセットの y の推定
estimated_y_prediction_all = []  # サブモデルごとの予測用データセットの y の推定結果を追加
for submodel_number in range(number_of_sub_datasets):
    # 説明変数の選択
    selected_x_columns = [
        x.columns[i] for i in selected_x_variable_numbers[submodel_number]
    ]
    selected_autoscaled_x_prediction = autoscaled_x_prediction.select(
        selected_x_columns
    )
    # 予測用データセットの y の推定
    estimated_y_prediction = submodels[submodel_number].predict(
        selected_autoscaled_x_prediction.to_numpy()
    )
    estimated_y_prediction = (
        estimated_y_prediction * y.std() + y.mean()
    )  # スケールをもとに戻します
    estimated_y_prediction_all.append(estimated_y_prediction)

# 予測用データセットの推定値の平均値
estimated_y_prediction_all_df = pl.DataFrame(estimated_y_prediction_all).transpose()
estimated_y_prediction = estimated_y_prediction_all_df.mean_horizontal().alias(
    "estimated_y"
)
estimated_y_prediction.to_frame().write_csv(
    "../output/estimated_y_prediction_ensemble_svr_gaussian.csv"
)

# 予測用データセットの推定値の標準偏差
std_of_estimated_y_prediction = estimated_y_prediction_all_df.std(1).alias(
    "std_of_estimated_y"
)
std_of_estimated_y_prediction.to_frame().write_csv(
    "../output/std_of_estimated_y_prediction_ensemble_svr_gaussian.csv"
)